# Micromobility in Minneapolis, MN: Spatiotemporal Prediction of Origins & Destinations

## Notebook 1: ETL and Data Wrangling

### Luke Zaruba, University of Minnesota - MGIS Program

##### March 20, 2023

*Micromobility has been a recent interest of transportation planners and urban residents alike. One of the most popular implementations of micromobility in Minneapolis has been through motorized scooters from companies like Lime (Uber), Bird, or Lyft. There are comprehensive datasets available containing trip information for all scooter trips within the City, but using the data for understanding travel patterns and user behaviors has largely remained untouched. The data could have immense value to planners, residents, and other stakeholders, to inform future decision-making and more adequately prepare for how the urban landscape will change to accommodate greater volumes of scooters and other forms of micromobility. My solution is to use the powerful techniques of spatial data science to uncover these patterns and estimate when and where users are traveling **to** and **from**.*

In this notebook, we will be demonstrating the extract, transform, and loading (ETL) process which prepares the data for analytics that we will run in another notebook, called *analysis.ipynb*. By the end of the notebook, we will go from two input datasets to having several Space Time Cubes created and ready for analysis.

In [1]:
# Ignore Warnings
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Import Libraries
import arcpy
import os

In [3]:
# Set Paths
working_gdb = r"C:\gitFiles\MicromobilityAnalytics\MicromobilityAPRX\MicromobilityAPRX.gdb"
data_path = r"C:\gitFiles\MicromobilityAnalytics\data\raw"

# Set as Working Envirnoment
arcpy.env.workspace = working_gdb

## Loading Data to File Geodatabase

First, we need to start by getting the data into a File Geodatabase, which will make the analysis easier (no need for file paths after setting a default workspace!).

In [4]:
# Copy Streets to GDB
arcpy.conversion.ExportFeatures(
    os.path.join(data_path, r"PW_Street_Centerline\PW_Street_Centerline.shp"),
    "streets_line"
)

# Copy Trips to GDB
arcpy.management.CopyRows(
    os.path.join(data_path, "Motorized_Foot_Scooter_Trips_2021.csv"),
    "trips_table"
)

<Result 'C:\\gitFiles\\MicromobilityAnalytics\\MicromobilityAPRX\\MicromobilityAPRX.gdb\\trips_table'>

## Cleaning the Street Centerlines

The spatial data in the scooter trips dataset is really just a street centerline ID, for both the origin and destination. So, in order to extract locations for the origins and destinations of the trips, we must first clean and transform the street centerlines dataset to prepare it for a join, which will attach an actual location to each trip's origin and destination.

In [5]:
# Create List of Fields
street_fields = [i.name for i in arcpy.ListFields("streets_line")]

# Create List of Fields to Drop
drop_street_fields = [i for i in street_fields if i not in ("GBSID", "OBJECTID_1", "Shape_Length", "Shape")]

# Drop Unnecessary Fields
arcpy.management.DeleteField(
    os.path.join(working_gdb, "streets_line"),
    drop_field = drop_street_fields,
    method = "DELETE_FIELDS"
)

<Result 'C:\\gitFiles\\MicromobilityAnalytics\\MicromobilityAPRX\\MicromobilityAPRX.gdb\\streets_line'>

In [6]:
# Calculate Centroid Coordinates
arcpy.management.CalculateGeometryAttributes(
    os.path.join(working_gdb, "streets_line"),
    geometry_property = "X CENTROID_X;Y CENTROID_Y",
    coordinate_system = arcpy.SpatialReference(4236),
    coordinate_format = "DD"
)

<Result 'C:\\gitFiles\\MicromobilityAnalytics\\MicromobilityAPRX\\MicromobilityAPRX.gdb\\streets_line'>

In [7]:
# Create Centroid Point Feature Class
arcpy.management.XYTableToPoint(
    os.path.join(working_gdb, "streets_line"),
    os.path.join(working_gdb, "street_centroids"),
    x_field = "X",
    y_field = "Y",
    coordinate_system = arcpy.SpatialReference(4326)
)

<Result 'C:\\gitFiles\\MicromobilityAnalytics\\MicromobilityAPRX\\MicromobilityAPRX.gdb\\street_centroids'>

## Cleaning the Scooter Trips

Now that the street centerlines have been cleaned and converted to point features, we can start cleaning up the scooter trips dataset. This process will include casting the temporal fields to date types, as well as filtering out records that do not start and end on streets (i.e., they could also potentially end on trails, but the number that do is minimal).

In [8]:
# Alter Start Time Field
arcpy.management.ConvertTimeField(
    "trips_table",
    input_time_field = "StartTime",
    input_time_format = "yyyy/MM/dd HH:mm:ss+00",
    output_time_field = "StartTime_Converted",
    output_time_type = "DATE"
)

<Result 'trips_table'>

In [9]:
# Alter End Time Field
arcpy.management.ConvertTimeField(
    "trips_table",
    input_time_field = "EndTime",
    input_time_format = "yyyy/MM/dd HH:mm:ss+00",
    output_time_field = "EndTime_Converted",
    output_time_type = "DATE"
)

<Result 'trips_table'>

In [10]:
# Restrict to Trips that Start & End on Streets
arcpy.conversion.ExportTable(
    os.path.join(working_gdb, "trips_table"),
    os.path.join(working_gdb, "trips_table_cleaned"),
    where_clause = "StartCenterlineType = 'street' AND EndCenterlineType = 'street'"
)

<Result 'C:\\gitFiles\\MicromobilityAnalytics\\MicromobilityAPRX\\MicromobilityAPRX.gdb\\trips_table_cleaned'>

## Joining Scooter Trips to Streets

Finally, we can join the spatial components of the street centerlines dataset to the trips dataset. We need to do it **twice** so that we have a table representing origins, and another representing destinations.

In this workflow, rather than using a standard join, we are making use of the [Join Features](https://pro.arcgis.com/en/pro-app/latest/tool-reference/geoanalytics-desktop/join-features.htm) tool in the GeoAnalytics Desktop toolbox. This tool works a little differently than a standard join because it is specifically intended to be used on big data and can run faster than standard joins, by making use of Apache Spark, a data processing engine which is built for working with big data.

In [11]:
# Join by Origin ID
arcpy.gapro.JoinFeatures(
    os.path.join(working_gdb, "trips_table"),
    os.path.join(working_gdb, "street_centroids"),
    os.path.join(working_gdb, "origins_table"),
    join_operation = "JOIN_ONE_TO_MANY",
    attribute_relationship="StartCenterlineID GBSID"
)

<Result 'C:\\gitFiles\\MicromobilityAnalytics\\MicromobilityAPRX\\MicromobilityAPRX.gdb\\origins_table'>

In [12]:
# Join by Destination ID
arcpy.gapro.JoinFeatures(
    os.path.join(working_gdb, "trips_table"),
    os.path.join(working_gdb, "street_centroids"),
    os.path.join(working_gdb, "destinations_table"),
    join_operation = "JOIN_ONE_TO_MANY",
    attribute_relationship="EndCenterlineID GBSID"
)

<Result 'C:\\gitFiles\\MicromobilityAnalytics\\MicromobilityAPRX\\MicromobilityAPRX.gdb\\destinations_table'>

## Convert Scooter Trips to Feature Classes

Now that we have two tables with all of our trip data, we simply need to convert the tables to feature classes. This is an easy process since the latitude and longitude have now been joined to the trips dataset.

In [13]:
# Convert Origins to Feature Class
arcpy.management.XYTableToPoint(
    os.path.join(working_gdb, "origins_table"),
    os.path.join(working_gdb, "origins_points"),
    x_field = "X",
    y_field = "Y",
    coordinate_system = arcpy.SpatialReference(4326)
)

# Convert Destinations to Feature Class
arcpy.management.XYTableToPoint(
    os.path.join(working_gdb, "destinations_table"),
    os.path.join(working_gdb, "destinations_points"),
    x_field = "X",
    y_field = "Y",
    coordinate_system = arcpy.SpatialReference(4326)
)

<Result 'C:\\gitFiles\\MicromobilityAnalytics\\MicromobilityAPRX\\MicromobilityAPRX.gdb\\destinations_points'>

## Creating Space Time Cubes

In order to run spatiotemporal analytics, the easiest way is to make use of Space Time Cubes (STCs). Space Time Cubes are NetCDF files that are specifically built for processing, analyzing, and visualizing spatiotemporal data.

We can create a space time cube from point fetaures by specifying how features should be placed into space-time bins. But first, we need to project the points to a project coordinate system.

After reprojecting the data, four STCs are created, at two different time steps (half-hour and daily) for both the origins and the destinations.

In [14]:
# Project to PCS for STC to Work Properly
arcpy.management.Project(
    "origins_points",
    os.path.join(working_gdb, "origins_points_utm15"),
    out_coor_system = arcpy.SpatialReference(26915),
    transform_method ="WGS_1984_(ITRF00)_To_NAD_1983",
    in_coor_system = arcpy.SpatialReference(4326)
)

arcpy.management.Project(
    "destinations_points",
    os.path.join(working_gdb, "destinations_points_utm15"),
    out_coor_system = arcpy.SpatialReference(26915),
    transform_method ="WGS_1984_(ITRF00)_To_NAD_1983",
    in_coor_system = arcpy.SpatialReference(4326)
)

<Result 'C:\\gitFiles\\MicromobilityAnalytics\\MicromobilityAPRX\\MicromobilityAPRX.gdb\\destinations_points_utm15'>

In [15]:
# Create Origin STCs (0.5-hour increment)
arcpy.stpm.CreateSpaceTimeCube(
    in_features = "origins_points_utm15",
    output_cube = r"C:\gitFiles\MicromobilityAnalytics\data\outputs\origins_stc.nc",
    time_field = "StartTime_Converted",
    time_step_interval = "30 Minutes",
    time_step_alignment = "END_TIME",
    aggregation_shape_type="HEXAGON_GRID"
)

# Create Destination STCs (0.5-hour increment)
arcpy.stpm.CreateSpaceTimeCube(
    in_features = "destinations_points_utm15",
    output_cube = r"C:\gitFiles\MicromobilityAnalytics\data\outputs\destinations_stc.nc",
    time_field = "EndTime_Converted",
    time_step_interval = "30 Minutes",
    time_step_alignment = "END_TIME",
    aggregation_shape_type="HEXAGON_GRID"
)

<Result 'C:\\gitFiles\\MicromobilityAnalytics\\data\\outputs\\origins_stc.nc'>

In [16]:
# Create Origin STCs (daily increment)
arcpy.stpm.CreateSpaceTimeCube(
    in_features = "origins_points_utm15",
    output_cube = r"C:\gitFiles\MicromobilityAnalytics\data\outputs\originsDaily_stc.nc",
    time_field = "StartTime_Converted",
    time_step_interval = "1 Day",
    time_step_alignment = "END_TIME",
    aggregation_shape_type="HEXAGON_GRID"
)

# Create Destination STCs (daily increment)
arcpy.stpm.CreateSpaceTimeCube(
    in_features = "destinations_points_utm15",
    output_cube = r"C:\gitFiles\MicromobilityAnalytics\data\outputs\destinationsDaily_stc.nc",
    time_field = "EndTime_Converted",
    time_step_interval = "1 Day",
    time_step_alignment = "END_TIME",
    aggregation_shape_type="HEXAGON_GRID"
)

<Result 'C:\\gitFiles\\MicromobilityAnalytics\\data\\outputs\\destinations_daily_stc.nc'>

## Time for Analysis!

Now that our ETL process is complete, we can make use of spatiotemporal analytical techniques to uncover more knowledge from the data.